In [ ]:
'''
load msq
load t5forconditionalgeneration
preprocess msq for t5
train t5?

run inference on jeopardy?
'''
!pip install transformers datasets
!pip install sentencepiece


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch

from datasets import load_dataset, load_metric, list_metrics
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollator, T5ForConditionalGeneration, T5TokenizerFast

from tqdm import tqdm

from typing import Dict, List, Optional

import dataclasses
from dataclasses import dataclass, field

import logging
import os
import sys

import numpy as np
import torch

from transformers import (
    T5ForConditionalGeneration, 
    T5Tokenizer, 
    EvalPrediction,
    DataCollator,
    Trainer,
    TrainingArguments)

In [4]:
import pandas as pd
train_mqr_df = pd.read_csv('drive/MyDrive/t5-data/download/mqr_with_category_train_filtered.csv')
test_mqr_df = pd.read_csv('drive/MyDrive/t5-data/download/mqr_with_category_test_filtered.csv')

In [31]:
train_mqr_df = train_mqr_df.drop('make',axis=1)

test_mqr_df = test_mqr_df.drop('make',axis=1)


In [32]:
train_mqr_df = train_mqr_df.drop('Unnamed: 0',axis=1)

test_mqr_df = test_mqr_df.drop('Unnamed: 0',axis=1)

In [33]:
train_mqr_df.count()

ill formed     96286
well formed    96286
category       96286
dtype: int64

In [34]:
#append category to well-formed
train_mqr_df['ill formed'] = train_mqr_df['category'] + ": " + train_mqr_df['ill formed']
train_mqr_df = train_mqr_df.drop('category', axis=1)

test_mqr_df['ill formed'] = test_mqr_df['category'] + ": " + test_mqr_df['ill formed']
test_mqr_df = test_mqr_df.drop('category', axis=1)

In [35]:
test_mqr_df.count()

ill formed     407
well formed    407
dtype: int64

In [36]:
train_mqr_df.count()

ill formed     96286
well formed    96286
dtype: int64

In [ ]:
checkpoint = "t5-small"
model = T5ForConditionalGeneration.from_pretrained(checkpoint)
tokenizer = T5TokenizerFast.from_pretrained(checkpoint)

In [38]:
import numpy as np
# def preprocess(example):
#   if example['ill formed'][-1] == "?":
#     inputs = example['ill formed'][:-1]
#     example['ill formed'] = example['ill formed'][:-1]

#   return example

#     # example['target_text'] = '%s ' % example['answers']['text'][0]
#   # model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
#   #return model_inputs

# # tokenize the examples
# def convert_to_features(example_batch):
#     input_encodings = tokenizer.batch_encode_plus(example_batch['ill formed'], pad_to_max_length=True, max_length=128)
#     target_encodings = tokenizer.batch_encode_plus(example_batch['well formed'], pad_to_max_length=True, max_length=32)

#     encodings = {
#         'input_ids': input_encodings['input_ids'], 
#         'attention_mask': input_encodings['attention_mask'],
#         'decoder_input_ids': target_encodings['input_ids'],
#         'decoder_attention_mask': target_encodings['attention_mask'],
#     }

#     labels = np.array(encodings['decoder_input_ids'], dtype='uint8')
#     # print(type(labels), labels[0:5])
#     labels[labels == 0] = -100

#     encodings['labels'] = labels.tolist()

#     return encodings

def convert_to_features(example_batch):

    input_encodings = tokenizer.batch_encode_plus(example_batch['ill formed'], 
                                                  max_length=128, 
                                                  add_special_tokens=True,
                                                  truncation=True, 
                                                  pad_to_max_length=True)
    
    target_encodings = tokenizer.batch_encode_plus(example_batch['well formed'], 
                                                   max_length=32, 
                                                   add_special_tokens=True,
                                                   truncation=True, pad_to_max_length=True)
                                                   
    encodings = {
        'input_ids': input_encodings['input_ids'], 
        'attention_mask': input_encodings['attention_mask'],
        'decoder_input_ids': target_encodings['input_ids'],
        'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings

def add_eos_examples(example):
  ill_prefix = 'context: generate question:  '
  well_prefix = 'question: '
  example['ill formed'] = ill_prefix + example['ill formed'] + " </s>"
  example['well formed'] = well_prefix + example['well formed'] + " </s>"
  return example

# def add_special_tokens(example):
#   example['questions'] = example['questions'].replace("{sep_token}", '<sep>')
#   return example

In [39]:
from datasets import Dataset
dataset_train = Dataset.from_pandas(train_mqr_df)
dataset_test = Dataset.from_pandas(test_mqr_df)

In [40]:
tokenized_train_dataset = dataset_train.map(add_eos_examples)
tokenized_train_dataset = tokenized_train_dataset.map(convert_to_features, batched=True)

tokenized_test_dataset = dataset_test.map(add_eos_examples)
tokenized_test_dataset = tokenized_test_dataset.map(convert_to_features, batched=True)



  0%|          | 0/96286 [00:00<?, ?ex/s]

  0%|          | 0/97 [00:00<?, ?ba/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


  0%|          | 0/407 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [41]:
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["ill formed", "well formed"])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["ill formed", "well formed"])

columns = ['input_ids', 'decoder_input_ids', 'attention_mask', 'decoder_attention_mask']

tokenized_train_dataset.set_format(type='torch', columns=columns)
tokenized_test_dataset.set_format(type='torch', columns=columns)

In [42]:
import torch
torch.save(tokenized_train_dataset, 'train_data.pt')
torch.save(tokenized_test_dataset, 'valid_data.pt')

In [55]:
# from transformers import DataCollatorForSeq2Seq
# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [43]:
@dataclass
class T2TDataCollator():
  def __call__(self, batch: List) -> Dict[str, torch.Tensor]:
    """
    Take a list of samples from a Dataset and collate them into a batch.
    Returns:
    A dictionary of tensors
    """
    
    input_ids = torch.stack([example['input_ids'] for example in batch])
    lm_labels = torch.stack([example['decoder_input_ids'] for example in batch])
    lm_labels[lm_labels[:, :] == 0] = -100 
    attention_mask = torch.stack([example['attention_mask'] for example in batch])
    decoder_attention_mask = torch.stack([example['decoder_attention_mask'] for example in batch])
    
    return {
        'input_ids': input_ids, 
        'attention_mask': attention_mask,
        'labels': lm_labels, 
        'decoder_attention_mask': decoder_attention_mask
    }

In [51]:
training_args = TrainingArguments(output_dir="/content/drive/MyDrive/t5-results", 
                                  per_device_train_batch_size=64, 
                                  per_device_eval_batch_size=16,
                                  gradient_accumulation_steps=16,
                                  learning_rate=1e-4, 
                                  num_train_epochs=10,
                                  logging_steps=100,
                                  run_name="end2end-questions-generation",
                                  evaluation_strategy="steps",
                                  dataloader_num_workers=4,
                                  load_best_model_at_end=True,
                                  save_steps=300,
                                  save_total_limit=2
                                  )

using `logging_steps` to initialize `eval_steps` to 100
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

In [53]:
logger = logging.getLogger(__name__)

# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    data_collator=T2TDataCollator()
)

# Training
trainer.train()
trainer.save_model('drive/MyDrive/t5-results/final-model-filtered')

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 96286
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 1024
  Gradient Accumulation steps =

Step,Training Loss,Validation Loss
100,1.486300,1.001426
200,1.372500,0.964072
300,1.334900,0.943505
400,1.313200,0.928900
500,1.294600,0.923954
600,1.285800,0.913895
700,1.276900,0.910533
800,1.271200,0.907546
900,1.264500,0.907201


***** Running Evaluation *****
  Num examples = 407
  Batch size = 16
***** Running Evaluation *****
  Num examples = 407
  Batch size = 16
***** Running Evaluation *****
  Num examples = 407
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/t5-results/checkpoint-300
Configuration saved in /content/drive/MyDrive/t5-results/checkpoint-300/config.json
Model weights saved in /content/drive/MyDrive/t5-results/checkpoint-300/pytorch_model.bin
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
***** Running Evaluation *****
  Num examples = 407
  Batch size =

In [ ]:
)

In [ ]:
# from transformers import AutoModelForSeq2SeqLM
# model = AutoModelForSeq2SeqLM.from_pretrained("drive/MyDrive/t5-results/checkpoint-2000")
model = T5ForConditionalGeneration.from_pretrained('drive/MyDrive/t5-results/checkpoint-900')
tokenizer = T5TokenizerFast.from_pretrained('t5-small')


In [ ]:
# !pip install ipython-autotime
# %load_ext autotime

In [13]:
from scipy.special import softmax

input_string = "This chilled leek & potato soup is traditionally topped with chopped chives."
category = 'Food and Drink: '


def hf_run_model(input_string, **generator_args):
  generator_args = {
  "max_length": 256,
  "num_beams": 4,
  "length_penalty": 1.5,
  "no_repeat_ngram_size": 3,
  "early_stopping": True,
  }
  input_string = "generate question: " + category + input_string + " </s>"
  input_ids = tokenizer.encode(input_string, return_tensors="pt")
  res = model.generate(input_ids.to(device), **generator_args)
  output = tokenizer.batch_decode(res, skip_special_tokens=True)
  output = [item.split("<sep>") for item in output]
  print(output)
  return output

hf_run_model(input_string)


[['question: What is this chilled leek & potato soup traditionally topped with chopped chives?']]


[['question: What is this chilled leek & potato soup traditionally topped with chopped chives?']]

In [ ]:
# input_ids = torch.tensor([tokenizer.encode("Sliced apples sautéed in butter are a classic filling for these French pancakes.")])

# language_id = tokenizer.lang2id["en"]  # 0
# langs = torch.tensor([language_id] * input_ids.shape[1])  # torch.tensor([0, 0, 0, ..., 0])

# # We reshape it to be of size (batch_size, sequence_length)
# langs = langs.view(1, -1)


AttributeError: ignored

In [14]:
##inference
import json
with open('drive/MyDrive/t5-data/download/sample_questions.json','r') as file:
  sample_questions = json.load(file)

In [15]:
sample_questions[0]

{'category': 'COMMON BONDS',
 'air_date': '1998-09-07',
 'question': "'A game of footsie,<br />a bribe,<br />a drunk person'",
 'value': '$500',
 'answer': 'Things that are done under the table',
 'round': 'Jeopardy!',
 'show_number': '3216'}

In [16]:
def hf_run_model(input_string, category, **generator_args):
  generator_args = {
  "max_length": 256,
  "num_beams": 4,
  "length_penalty": 1.5,
  "no_repeat_ngram_size": 3,
  "early_stopping": True,
  }
  input_string = "generate question: " + category + input_string + " </s>"
  input_ids = tokenizer.encode(input_string, return_tensors="pt")
  res = model.generate(input_ids.to(device), **generator_args)
  output = tokenizer.batch_decode(res, skip_special_tokens=True)
  output = [item.split("<sep>") for item in output]
  return output


for sam in sample_questions:
  question, category = sam['question'], sam['category']
  pred = hf_run_model(question, category)
  sam['predicted_question'] = pred




In [17]:
sample_questions[0:10]

[{'category': 'COMMON BONDS',
  'air_date': '1998-09-07',
  'question': "'A game of footsie,<br />a bribe,<br />a drunk person'",
  'value': '$500',
  'answer': 'Things that are done under the table',
  'round': 'Jeopardy!',
  'show_number': '3216',
  'predicted_question': [['question: What is the difference between a game of footsie,br />a bribe and a drunk person?']]},
 {'category': 'CONGRESSIONAL MISDEMEANORS',
  'air_date': '2005-06-29',
  'question': "'Senator Benjamin Tappan was censured in 1844 for leaking information about the annexation of this to the Union'",
  'value': '$2000',
  'answer': 'Texas',
  'round': 'Double Jeopardy!',
  'show_number': '4808',
  'predicted_question': [["question: What was Benjamin Tappan's censure in 1844 for leaking information about the annexation of this to the Union?"]]},
 {'category': 'THE CHAIN GANG',
  'air_date': '1998-05-22',
  'question': '\'In any of this chain of theme parks, like the one "Over Texas", you might run into Looney Tunes ch

In [20]:

json_object = json.dumps(sample_questions, indent=4)
 
# Writing to sample.json
with open("drive/MyDrive/t5-results/sample_questions_with_predicted.json", "w") as outfile:
    outfile.write(json_object)



In [47]:
import numpy as np
temp_df = test_mqr_df
temp_df['make'] = temp_df['well formed'].str.split(' ').str[0]
temp_df.head(10)

,ill formed,well formed,category,make
0,Are only simetric matrices definite?,Are only symmetric matrices definite?,math,Are
1,Why the pope can't have children?,Why can't the Pope have children?,christianity,Why
2,Bonding wireless networks,How to bond wireless networks?,askubuntu,How
3,Measuring Individual Team Member Performance,How to measure a team member's performance?,pm,How
4,Does ChaCha20-Poly1305 needs random nonce?,Does ChaCha20-Poly1305 need random nonce?,crypto,Does
5,Time Magazine aiding Al Capone?,Did Time magazine aid Al Capone?,history,Did
6,Support Point for Model Wing Tested inside Win...,Where should the support points be for testing...,aviation,Where
7,What is Touch Word™?,What is a Touch Word™?,puzzling,What
8,Authenticating baseball paraphernalia,Who authenticates baseball paraphernalia?,sports,Who
9,Did Obama secretely visit Antarctica?,Did Obama secretly visit Antarctica?,skeptics,Did


In [48]:
qs = ['what', 'where', 'who', 'when', 'whose', 'which']
temp_df = temp_df[temp_df['make'].str.contains('What|Where|Who|When|Whose|Which', regex=True)]

In [49]:
temp_df.count()

ill formed     407
well formed    407
category       407
make           407
dtype: int64

In [50]:
temp_df.head(50)

,ill formed,well formed,category,make
6,Support Point for Model Wing Tested inside Win...,Where should the support points be for testing...,aviation,Where
7,What is Touch Word™?,What is a Touch Word™?,puzzling,What
8,Authenticating baseball paraphernalia,Who authenticates baseball paraphernalia?,sports,Who
14,Meaning using have/had been,What is the meaning of using have/had been?,ell,What
24,What is an Telemetry?,What is Telemetry?,electronics,What
26,directed random walk boundary hitting,Where does directed random walk hit the boundary?,math,Where
27,What does unphyisical quantity mean?,What does unphysical quantity mean?,physics,What
31,Beginner role and race for nethack?,What's a good role and race for a Nethack begi...,gaming,What's
41,Paladin's source of spellcasting ability,What is the source of a Paladin's spellcasting...,rpg,What
44,MATLAB Implementation of TCAS Algorithm,Where could I find a MATLAB implementation of ...,aviation,Where


In [51]:
temp_df.drop('make', axis = 1)

,ill formed,well formed,category
6,Support Point for Model Wing Tested inside Win...,Where should the support points be for testing...,aviation
7,What is Touch Word™?,What is a Touch Word™?,puzzling
8,Authenticating baseball paraphernalia,Who authenticates baseball paraphernalia?,sports
14,Meaning using have/had been,What is the meaning of using have/had been?,ell
24,What is an Telemetry?,What is Telemetry?,electronics
...,...,...,...
2068,Xbox: Where are language settings in Skyrim?,Where are the language settings in the Xbox 36...,gaming
2072,What are actually complex numbers?,"What are complex numbers, actually?",math
2075,Significance of extension tube lengths,What is the significance of the typical extens...,photo
2090,When a player is flagged as cheater?,When is a player flagged as a cheater?,gaming


In [52]:
compression_opts = dict(method='zip',
                        archive_name='mqr_with_category_filtered.csv')  
temp_df.to_csv('mqr_with_category_filtered.zip', index=False,
          compression=compression_opts)  

In [53]:
temp_df.to_csv('drive/MyDrive/t5-data/download/mqr_with_category_test_filtered.csv')  